## Code for interpolating daily rainfall station data. The daily station data has length of about 30+ years and it is used to make rasters of daily interpolated values using (1) inverse distance weighting method and (2) splines functions.  Thur's Digital elevation map (DEM) is used to incorporate the influence of height on interpolated rainfall fields.

In [ ]:
import os
import sys
import subprocess
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["/Applications/GRASS-7.3.app/Contents/MacOS/grass73", "--config", "path"])
gisbase = gisbase.strip().decode()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

In [ ]:
import grass.script as gs
import grass.script.setup as gsetup

In [ ]:
# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "/Users/lisac/grassdata", "Thur_dem25_utm", "Practice")

In [ ]:
gs.message('Current GRASS GIS 7 environment:')
print(gs.gisenv())

### Arranging the data for daily interpolations

In [ ]:
import glob
import pandas as pd

rainfall_interp_input_df = pd.DataFrame([])

for csv_file in sorted(glob.glob(base+'weather_data/processed/*.csv')):
    name = csv_file.split('/')[-1][:-4]
    csv_file_temp = pd.read_csv(csv_file, index_col=[0], parse_dates=[0], usecols=[0,2], header=None, names=['time', name], skiprows=1)
    rainfall_interp_input_df = pd.concat([rainfall_interp_input_df, csv_file_temp], axis=1)

### Employing GrassGIS from python in order to make: (1) data transformation from vectorized format, (2) daily interpolations based on distance weighting and splines functions, (3) resulting daily interpolations in resterized format

In [ ]:
i=0

# Thur rainfall station data
base = '/Users/lisac/Documents/Data_analysis/Thur/'

# weather metadata to stick rainfall data to it for each day, and use for interpolation
stations = pd.read_csv(base+'interpolations/weather_stations_dem25_utm.txt')

# all possible rainfall data
rainfall_interp_input_df = pd.read_csv(base+'interpolations/rainfall_stations_input_for_interp.txt', index_col=[0], parse_dates=[0])
rainfall_interp_input_df = 0.1*rainfall_interp_input_df #transform to [cm] of rainfall

# resulting raster shape, used later for initilizing the array to store yearly values in
raster_shape = (2260, 2466)
years = np.arange(1970, 1980, 1)

for one_year in years:
    begin_date = f'{one_year}-01-01'
    end_date = f'{one_year}-12-31'


    rainfall_one_year = rainfall_interp_input_df[begin_date:end_date].copy()
    rainfall_interp_all_days_ar = np.empty((len(rainfall_one_year), *raster_shape))

# connection to GrassGIS
    for i in range(len(rainfall_one_year)):
        print('day number:', i)
        stations['rain [mm]'] = rainfall_one_year.iloc[i].values
        stations.to_csv('new' + str(1) + '.txt')
        gs.run_command('v.in.ascii', overwrite=True, input = 'new1.txt', output='stations_rain_vector',
                   separator=',', skip=1, x=2, y=3, z=4, cat=1)
        gs.run_command('v.surf.idw', overwrite=True, input = 'stations_rain_vector', output='output',
                   column='dbl_3', npoints=12, power=2.0)

#  spline interpolation       
#        gs.run_command('v.vol.rst', overwrite=True, input = 'stations_rain_vector', cross_input='Chile_90m_dem@PERMANENT',
#                cross_output=out_number[i], wcolumn='dbl_3', tension=10., smooth=0.5, segmax=700, npmax=700, npmin=10)
        gs.run_command('r.out.ascii', overwrite=True, input = 'output', output='/Users/lisac/Documents/Grass and jupyter stuff/rain_interpolation_cross_ascii.csv')
        rain_interpolation_1day = pd.read_csv('/Users/lisac/Documents/Grass and jupyter stuff/rain_interpolation_cross_ascii.csv',
                                               header=None, delimiter=' ', skiprows=6)
        rain_interpolation_1day.drop(columns=len(rain_interpolation_1day.columns)-1, inplace=True)
        rainfall_interp_all_days_ar[i,:,:] = np.array(rain_interpolation_1day)
    
       


    rounded = np.round(rainfall_interp_all_days_ar, decimals=2)
    rounded = 100*rounded
    rounded_int = rounded.astype('int16')
    np.save(base+f'interpolation_idw_dem25_utm_{one_year}', rounded_int)